# **Partie 3 : Modélisation**

Prédiction de l'occurence d'un feu de forêt et de la surface brûlée grâce à l'apprentissage supervisé. 


## **A. Prédiction du déclenchement d'un feu de forêt**

L'objectif de cette partie est de prédire les zones dans lesquelles un feu de forêt est propice de se déclencher dans les minutes à venir. On s'aide pour cela des relevés météorologiques pris en temps réel. 



Le problème auquel nous sommes confrontés est celui d'une classification en deux catégories (va-t-il y avoir un départ de feu ou non ?). Nous disposons des données des relevés météorologiques et feux passés pour entraîner notre algorithme. 


Nous avons alors sélectionné deux modèles adaptés : 


1.   Decision Tree Classifier
2.   Random Forest Regression
3.   Réseaux de neurones

## Premier essai des modèles avec les valeurs météorologiques brutes

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

On utilise la base de données obtenue dans la partie 1, dont on ne va garder plus que les variables météorologiques et la variable binaire *Départ de feu*.

In [3]:
df = pd.read_csv('data.zip', low_memory = False)
df = df.iloc[:, [0,21,22,23,24]]  
df = df.dropna()

### **Préparation des bases d'apprentissage et de test**

On sépare de façon aléatoire la base de données obtenue dans la partie 1 en deux bases : 80% servira à l'entraînement de l'algorithme (base train) et 20% à observer sa précision (base test). 

In [4]:
train, test = train_test_split(df, test_size=0.2, random_state=1, stratify = df['Départ de feu'])

 On vérifie que les proportions des classes sont bien respectées.

In [5]:
print("La proportion de départ de feux dans la base d'entraînement est de " + str(round(train["Départ de feu"].mean() * 10**(4))/10**2) + '%')
print("Elle est également de " + str(round(test["Départ de feu"].mean() * 10**(4))/10**2) + '% dans la base de test')

La proportion de départ de feux dans la base d'entraînement est de 1.61%
Elle est également de 1.61% dans la base de test


Ensuite, on spécifie pour chaque base la matrice X des variables prédictives (les données météorologiques) et le vecteur Y de la variable cible (*Départ de feu*).

Le vecteur Y défini 2 classes : 


*   Une observation est de classe 0 s'il n'y a pas de départ de feux, i.e. "Départ de feu" == 0.
*   Une observation est de classe 1 s'il y a départ de feu, i.e. "Départ de feu" == 1.

On appelle Vrai Négatif $-$ VN $-$ (resp. Vrai Positif $-$ VP $-$) une observation de classe 0 (resp. de classe 1) qui est reconnue par le modèle comme étant de classe 0 (resp. de classe 1). 

Un Faux Négatif $-$ FN $-$ (resp. Faux Positif $-$ FP $-$) est une observation prédite de classe 0 alors qu'elle est de classe 1 (resp. prédite de classe 1 alors qu'elle est de classe 0).


In [6]:
X_train = train.drop(['Départ de feu'], axis = 1).to_numpy()
Y_train = train[['Départ de feu']].to_numpy()

X_test = test.drop(['Départ de feu'], axis = 1).to_numpy()
Y_test = test[['Départ de feu']].to_numpy()

Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])

### **Modèle 1 : l'arbre de décision**

**Entraînement**

On entraîne le modèle sur la base d'entraînement.

In [7]:
#instanciation de l'arbre
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

#construction de l'arbre
tree = tree.fit(X_train, Y_train)

**Evaluation**

On évalue les performances prédictives de l'arbre en le testant sur la base de test, puis en comparant les prédictions avec la réalité. 

*   Distribution de la prédiction

In [8]:
tree_predict_test = tree.predict(X_test)

#distribution 
tree_predict = np.unique(tree_predict_test,return_counts=True)
print(tree_predict)
print('La classe ' + str(tree_predict[0][0]) + ' a été assignée ' + str(tree_predict[1][0]) + ' fois.')

(array([0, 1]), array([401140,   5159]))
La classe 0 a été assignée 401140 fois.


In [9]:
print("Notre modèle prédit ainsi qu'il y a parmi la base de test " + str(tree_predict[1][0]) + " observations \
ne correspondant pas à un feu de forêt (classe " + str(tree_predict[0][0]) + ").")
print("Il prédit que " + str(tree_predict[1][1]) + " observations \
correspondent à un feu de forêt (classe " + str(tree_predict[0][1]) + ").")

Notre modèle prédit ainsi qu'il y a parmi la base de test 401140 observations ne correspondant pas à un feu de forêt (classe 0).
Il prédit que 5159 observations correspondent à un feu de forêt (classe 1).


*   Calcul de la matrice de confusion

IMAGE MANQUANTE

Dans notre cas, la i-ème ligne est la ligne des "Départ de feu" = i et la i-ème colonne est la "Prédiction de Départ de feu" = i

In [10]:
#calcul de la matrice de confusion 
print('Matrice de confusion')
confu_tree = confusion_matrix(Y_test,tree_predict_test)
print(confu_tree)  

Matrice de confusion
[[395927   3811]
 [  5213   1348]]


Les lignes et les colonnes sont données par ordre croissant. La ligne donne la classe observée (c'est-à-dire la réalité), la colonne la classe prédite (par le modèle). Par exemple : 


In [11]:
print(str(confu_tree[0][0]) + " observations sans feu ont été prédites à raison comme ne correspondant pas à un feu de forêt.")
print(str(confu_tree[1][0]) + " observations avec feu ont été prédites à tort comme ne correspondant pas à un feu de forêt.")

395927 observations sans feu ont été prédites à raison comme ne correspondant pas à un feu de forêt.
5213 observations avec feu ont été prédites à tort comme ne correspondant pas à un feu de forêt.


La matrice de confusion nous permet déjà d'observer qu'une très grande proportion des observations correspondant à un feu de forêt ne sont pas reconnues comme telle par le modèle. 

*   Indicateurs de précision

In [12]:
#taux de reconnaissance
print('le taux de reconnaissance est :', accuracy_score(Y_test,tree_predict_test))

#taux d'erreur 
print("le taux d'erreur est :", 1.0 - accuracy_score(Y_test,tree_predict_test))

le taux de reconnaissance est : 0.9777897558202211
le taux d'erreur est : 0.022210244179778904


Où le taux de reconnaissance est défini par : 

$ ( VP + VN ) / Nombre total d'observations $ 

Le taux de reconnaissance de notre modèle est très bon. Néanmoins, il nous faut encore regarder la répartition de faux-négatifs (déclarés de classe 0 à tort) et de faux-positifs (déclarés de classe 1 à tort). On utilise pour cela le rapport de classification.

In [13]:
print('Rapport de prédiction sur la base de test :')
print(classification_report(Y_test, tree_predict_test))

Rapport de prédiction sur la base de test :
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    399738
           1       0.26      0.21      0.23      6561

    accuracy                           0.98    406299
   macro avg       0.62      0.60      0.61    406299
weighted avg       0.98      0.98      0.98    406299



**Comment lire le rapport de prédiction ?**

*   *Precision : quelle proportion des prédictions est correcte ?*

Pour chaque classe, la colonne *precision* indique la proportion d'observations prédites à raison dans cette classe. 


Alors, 

$precision_0 = VN / ( VN + FN) $

$precision_1 = VP / ( VP + FP ) $ 



*   *Recall : quelle proportion des cas est detectée ?*

Pour chaque classe, la colonne *Recall* indique la proportion d'observations de cette classe qui ont été correctement prédites. 

$recall_0 = VN / ( VN + FP ) $

$recall_1 = VP / ( VP + FN ) $ 





*   *F1-score* 

Le *f1 score* est la moyenne harmonique de *precision* et *recall*. 

$f1score_i = 2 * (recall_i * precision_i) / (recall_i + precision_i)$ 



**Interprétation**

Le modèle semblait être satisfaisait car il avait un faible taux d'erreur. 
Néanmoins, il prédit très mal les observations de classe 1 : à peine un cinquième d'entre elles sont reconnues par l'arbre de décision. La faible proportion d'observations de classe 1 dans la base de données (1,6%) explique que le modèle conserve une très grande précision malgré ce défaut majeur.

Le modèle de l'arbre de décision n'est donc pas satisfaisant. 

### **Modèle 2 : Random Forest**

Ce modèle combine de nombreux arbres de décision, ce qui devrait le rendre plus performant que le premier modèle. 



**Entraînement**

In [14]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc = rfc.fit(X_train, Y_train)

#Attention : cette cellule met cinq bonnes minutes à tourner 

**Evaluation**

In [15]:
rfc_predict_test = rfc.predict(X_test)

In [16]:
#calcul de la matrice de confusion 
print('Matrice de confusion')
confu_rfc = confusion_matrix(Y_test,rfc_predict_test)
print(confu_rfc)   

#taux de reconnaissance
print('le taux de reconnaissance est :', accuracy_score(Y_test,rfc_predict_test))

#taux d'erreur 
print("le taux d'erreur est :", 1.0 - accuracy_score(Y_test,rfc_predict_test))

Matrice de confusion
[[398023   1715]
 [  5319   1242]]
le taux de reconnaissance est : 0.9826876266001147
le taux d'erreur est : 0.01731237339988534


In [17]:
print('Rapport de prédiction sur la base de test :')
print(classification_report(Y_test, rfc_predict_test))

Rapport de prédiction sur la base de test :
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    399738
           1       0.42      0.19      0.26      6561

    accuracy                           0.98    406299
   macro avg       0.70      0.59      0.63    406299
weighted avg       0.98      0.98      0.98    406299



Le même problème qu'au modèle 1 apparaît : la plupart des observations correspondant à un feu de forêt ne sont pas reconnues par le modèle. Du fait de l'écrasante majorité d'observations ne correspond pas à un feu de forêt, le modèle semble très précis, mais il est dans les faits très peu performant. 

En conclusion, les modèles de Machine Learning que nous avons testés sont inefficaces pour prédire le déclenchement d'un feu de forêt. 

On peut évoquer plusieurs raisons à cela : 

*   Il y a une bien trop faible proportion d'observations correspondant à un feu de forêt. 
*   Les variables étudiées ne suffissent pas à expliquer le déclenchement d'un feu de forêt : il ne suffit pas qu'il fasse la bonne température, couplée avec la bonne humidité (ou avec la bonne vitesse du vent, le bon niveau de précipitation, etc) pour qu'un feu parte. 


### **Modèle 3 : Réseaux de neurones**

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import activations
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error


In [19]:
df = pd.read_csv('data.csv').dropna(subset = ['Température (°C)', 'Précipitations (3 heures)', 'Humidité (%)', 'Vitesse du vent (m/s)'])
df_feu = df[df['Départ de feu'] == 1]
df_feu = df_feu.drop(columns = ['Départ de feu'])
df_feu = df_feu.sort_values(['Date de première alerte'])
df_feu = df_feu.reset_index(drop = True)
df_feu.head()

FileNotFoundError: ignored

In [ ]:
df_feu.dtypes

In [ ]:
values = df_feu.values
values

In [ ]:
x = values[:,[22,23]].astype(float)
y = values[:,8].astype(float)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.3, random_state = 10)

In [ ]:
x_train.shape

In [ ]:
model = Sequential()

model.add(Dense(500, input_dim = 2, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(1))

model.compile(loss = "mean_squared_error" , optimizer = "adam", metrics = ["mean_squared_error"])

model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs = 20, verbose = 1, validation_data=(x_test, y_test))

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['loss'], lw=3, label='Training set')
plt.plot(history.epoch, history.history['val_loss'], lw=3, label='Test set')
plt.legend(fontsize=14)
plt.title('Loss of the FFNN', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('MAE', fontsize=14)
plt.tight_layout()

In [ ]:
pred_train = model.predict(x_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred = model.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

In [ ]:
confusion_matrix(y_true, y_pred)

## Deuxième essai : création de nouvelles variables

Comme vu précédemment, le faible nombre de variables considérées est une des raisons pour lesquelles nos modèles ne sont pas satisfaits. 

Dans cette partie, nous tâcherons de créer de nouvelles variables qui témoigneront de la propension d'une observation à être un feu de forêt grâce à sa localisation et au mois considéré. Notre étude prend désormais en compte des variables géographiques (et non juste météorologiques).

### **Création des variables**

In [20]:
from datetime import datetime
df = pd.read_csv('data.zip', low_memory = False)

In [21]:
df_feu = df[df["Départ de feu"] == 1]

**Création d'une variable sur le mois** 

L'on va ici chercher à créer une variable quantitative qui rende compte de la "prédisposition" d'un mois à connaître des feux de forêts. Comme on a pu le voir précédemment, il y a empiriquement plus de feux en août qu'en octobre. On cherche donc une variable qui permette d'en témoigner. 

On crée ainsi une variable *Mois propice* qui à chaque observation donne le pourcentage des feux des années précédentes (dans la limite de nos données, c'est-à-dire l'année 2006) s'étant produits durant ce mois de l'année. Plus ce chiffre est élevé, plus ce mois est à risque.

Cette variable s'appuie exclusivement sur les données des années précédentes. La valeur attribuée à chaque mois varie également d'une année à l'autre, car elle prend en compte les feux de l'année qui vient de s'écouler. Ce comptage est intéressant car il gagne en fiabilité d'années en années. 


In [22]:
répartition = []

for année in range(2007,2020) :

  mois, nb_mois_previous_years = np.unique(df_feu['Date de première alerte'][df_feu.Année < année].str[5:7], return_counts=True)
  répartition_previous_years = nb_mois_previous_years / sum(nb_mois_previous_years)

  répartition.append([année,répartition_previous_years])

répartition[0:3]

[[2007, array([0.01557839, 0.03579715, 0.05502154, 0.09214451, 0.09048724,
         0.1528008 , 0.25223732, 0.18362612, 0.06463374, 0.02320186,
         0.02552204, 0.00894929])],
 [2008, array([0.02895404, 0.03184944, 0.07871878, 0.09156714, 0.06858487,
         0.11653999, 0.2062975 , 0.15635179, 0.09989142, 0.06424177,
         0.03999276, 0.0170105 ])],
 [2009, array([0.03337488, 0.06051793, 0.08613765, 0.08932281, 0.06093339,
         0.09735494, 0.19055532, 0.15939621, 0.09901676, 0.07519734,
         0.0340673 , 0.01412547])]]

Le tableau *répartition* indique pour chaque année la proportion par mois des feux déclarés les années précédentes (depuis 2006). 

In [23]:
répartition[4]

[2011, array([0.02136822, 0.0519987 , 0.11594085, 0.08409165, 0.05492363,
        0.09140396, 0.190039  , 0.17907052, 0.10814105, 0.06857329,
        0.02445564, 0.0099935 ])]

Par exemple, entre 2006 et 2010, 2,1% des feux se sont produits durant les mois de janviers, alors que 19% des feux entre 2006 et 2010 ont été en juillet. 

In [24]:
def mois_propice(date) : 
    "renvoie la proportion des feux déclenchés ce mois-ci les années précédentes"
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    month, year = date.month, date.year

    if year > 2006 : 
      return répartition[year-2007][1][month-1]

In [25]:
df['Mois propice'] = df['Date'].apply(lambda row: mois_propice(row))

**Création d'une variable sur la station** 

On crée maintenant une variable qui témoigne de la "prédisposition" d'une zone géographique (représentée par une station) à connaître des feux de forêts. 

On raisonne pareillement que pour la variable *Mois propice* en termes de pourcentage de feux des années précédentes observée par station. 


On ne peut en effet pas : 


*   Compter le nombre de feux par station dans la base, car lorsqu'on utilisera les méthodes de prédiction sur une certaine observation (à une certaine date), cela voudrait dire que l'on utilise les données de dates ultérieures, et donc qu'on connaît l'avenir. 


*   Compter le nombre de feux par station qui se sont produits avant l'observation. Les valeurs prises par une station seraient différentes d'une année à l'autre, et la variable n'aurait alors plus de sens. (Par exemple, on peut imaginer qu'il y ait autant de feu en Haute-Corse en 2006 qu'en Bretagne entre 2006 et 2018 : la valeur prise par la variable *Zone propice* pour une observation en Corse en 2007 serait égale à celle prise par une observation en Bretagne en 2019, ce qui ne rendrait pas bien compte de la réalité lorsqu'on utilisera les techniques de prédiction. 

Considérer les données sur toutes les années précédentes permet de créer une variable plus fiable que si l'on utilisait que celle d'une année précédente.

In [26]:
répartition_zone = []

for année in range(2007,2020) :

    zone, nb_zone = np.unique(df_feu['id_station'][df_feu.Année < année], return_counts=True)
    prop_zone = nb_zone / sum(nb_zone)
    prop_zone_previous = [ [zone[i], prop_zone[i]] for i in range(0,len(zone)) ]
    répartition_zone.append([année,prop_zone_previous])

*répartition_zone* est un tableau donnant, pour chaque année, la proportion de feux s'étant déclarée dans chaque station les années précédentes

In [27]:
#Exemple : la répartition pour l'année 2007
répartition_zone[0][1]

[[7110, 0.0006629101756711965],
 [7130, 0.0019887305270135896],
 [7139, 0.004971826317533974],
 [7149, 0.003646005966191581],
 [7207, 0.0016572754391779914],
 [7222, 0.00033145508783559825],
 [7240, 0.0029830957905203847],
 [7255, 0.0016572754391779914],
 [7280, 0.0009943652635067948],
 [7299, 0.0016572754391779914],
 [7314, 0.003977461054027179],
 [7335, 0.0029830957905203847],
 [7434, 0.011269472986410341],
 [7460, 0.0019887305270135896],
 [7471, 0.008286377195889956],
 [7481, 0.010275107722903546],
 [7510, 0.12595293337752733],
 [7535, 0.03513423931057342],
 [7558, 0.01358965860125953],
 [7577, 0.0573417301955585],
 [7591, 0.005634736493205171],
 [7607, 0.07093138879681803],
 [7621, 0.00033145508783559825],
 [7627, 0.013258203513423931],
 [7630, 0.006297646668876367],
 [7643, 0.04441498176997017],
 [7650, 0.07590321511435201],
 [7661, 0.060656281073914485],
 [7690, 0.07225720914816043],
 [7747, 0.06529665230361285],
 [7761, 0.1537951607557176],
 [7790, 0.13921113689095127],
 [61968,

In [28]:
def zone_propice(date,station) : 
    "renvoie la proportion des feux déclenchés dans cette zone les années précédentes"

    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    year = date.year
 
    if year > 2006 : 
      ligne = [i for i in range(0,len(répartition_zone[year-2007][1])) if répartition_zone[year-2007][1][i][0] == station]
      #liste des lignes correspondant à l'année et à la station 
      if len(ligne) != 0 :
      #len(ligne) == 1 si la station n'est pas dégénérée (existe dans les données)
        return répartition_zone[year-2007][1][ligne[0]][1]

In [29]:
df['Zone propice'] = df.apply(lambda row: zone_propice(row['Date'], row['id_station']), axis=1)

In [30]:
#Exemple d'une station Corse
df[df["id_station"] == 7761	][["Date","Zone propice"]].sample(5) 

,Date,Zone propice
1634710,2014-04-15 21:00:00,0.142842
1632909,2013-09-03 18:00:00,0.142702
1637371,2015-03-09 00:00:00,0.142891
1634337,2014-02-28 15:00:00,0.142842
1612181,2006-09-30 03:00:00,NaN


Le raisonnement que l'on va maintenant mener est identique à celui que nous avons eu dans le premier essai. Nous allons comparer les résultats des deux essais, et on s'attend à ce que ce deuxième essai, qui prend en compte davantage de variables, ait de meilleurs résultats. 

### **Préparation des bases d'apprentissage et de test**

In [32]:
df = df.iloc[:, [0,17,18,19,21,22,23,24,25,26]]  
df = df.dropna()

In [34]:
df.head()

,Départ de feu,Latitude station,Longitude station,Altitude station,Température (°C),Précipitations (3 heures),Humidité (%),Vitesse du vent (m/s),Mois propice,Zone propice
69920,0,50.57,3.0975,47,7.7,-0.1,86.0,4.1,0.024042,0.000042
69921,0,50.57,3.0975,47,5.4,0.0,87.0,3.3,0.024042,0.000042
69922,0,50.57,3.0975,47,2.7,0.0,96.0,1.4,0.024042,0.000042
69923,0,50.57,3.0975,47,2.7,-0.1,97.0,0.9,0.024042,0.000042
69924,0,50.57,3.0975,47,6.3,0.0,97.0,2.9,0.024042,0.000042


Le dataframe que nous considérons maintenant comporte davantage de variables que celui utilisé précédemment : on a des informations sur la localisation de la station (latitude, longitude et altitude) et sur la prédisposition du mois et de la zone à être victime de feux de forêt. 

In [35]:
train, test = train_test_split(df, test_size=0.2, random_state=1, stratify = df['Départ de feu'])

In [36]:
X_train = train.drop(['Départ de feu'], axis = 1).to_numpy()
Y_train = train[['Départ de feu']].to_numpy()

X_test = test.drop(['Départ de feu'], axis = 1).to_numpy()
Y_test = test[['Départ de feu']].to_numpy()

Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])

### **Modèle 1 : l'arbre de décision**

**Entraînement**

In [37]:
#instanciation de l'arbre
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()

#construction de l'arbre
tree = tree.fit(X_train, Y_train)

**Evaluation**

In [41]:
tree_predict_test = tree.predict(X_test)

Comme l'on a pu le voir précédemment, les taux de reconnaissance et d'erreur du modèle peuvent être trompeurs du fait de la faible proportion d'observations correspondant à un feu de forêt. On s'intéressera donc principalement au rapport de prédiction. 

In [40]:
print('Rapport de prédiction sur la base de test :')
print(classification_report(Y_test, tree_predict_test))

Rapport de prédiction sur la base de test :
              precision    recall  f1-score   support

           0       0.99      0.98      0.98    282495
           1       0.29      0.37      0.32      5951

    accuracy                           0.97    288446
   macro avg       0.64      0.67      0.65    288446
weighted avg       0.97      0.97      0.97    288446



On observe encore une fois une très bonne performance du modèle sur les observations ne correspondant pas à un feu de forêt. 

En ce qui concerne les observations correspondant à un feu de forêt (*Départ de feu* == 1), le modèle affiche de meilleures performances en précision et en recall. Cela est encourageant, néanmoins nous sommes encore loin d'une prédiction satisfaisante. 

### **Modèle 2 : Random forest**

Le modèle *Random Forest* est censé nous livrer de meilleurs résultats qu'un arbre de décision seul.

**Entraînement**

In [42]:
rfc = RandomForestClassifier()
rfc = rfc.fit(X_train, Y_train)

**Evaluation**

In [43]:
rfc_predict_test = rfc.predict(X_test)

In [45]:
#calcul de la matrice de confusion 
print('Matrice de confusion')
confu_rfc = confusion_matrix(Y_test,rfc_predict_test)
print(confu_rfc)  

Matrice de confusion
[[282030    465]
 [  4240   1711]]


In [44]:
print('Rapport de prédiction sur la base de test :')
print(classification_report(Y_test, rfc_predict_test))

Rapport de prédiction sur la base de test :
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    282495
           1       0.79      0.29      0.42      5951

    accuracy                           0.98    288446
   macro avg       0.89      0.64      0.71    288446
weighted avg       0.98      0.98      0.98    288446



Notre travail commence à porter ses fruits : 

Le modèle *Random Forest* a une excellente performance sur les observations ne correspondant pas à un feu de forêt. 

Concernant les observations correspondant à un feu de forêt, la précision s'est grandement améliorée et commence à devenir intéressante. La part de faux positifs a diminué ( $ = 1 - precision_1 $), comme on peut l'obverser sur la matrice de confusion (cellule 1,1). Le modèle arrive à mieux reconnaître les observations ne correspondant pas à un feu de forêt. 

Néanmoins, $recall_1$ demeure faible : le modèle a des difficultés à reconnaître lorsqu'une observation correspond à un feu de forêt. 




### **Modèle 3 :**